In [1]:
#from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
import pickle
import math
from six.moves import range
from six.moves.urllib.request import urlretrieve

from twodlearn.tf_lib.Feedforward import LinearLayer

import sys
working_dir= '/home/marinodl/dl_projects/nlp/sentiment_analysis/sa_by_word/'
sys.path.insert(0, working_dir)

#### Configuration Variables

In [2]:
allow_valid= True
Valid_Percentage= 0.1 # i.e. 10% of the data will be used for validation

pos_net_name= '_pos'
neg_net_name= '_neg'

activation_function='tanh'
num_nodes = [300, 300, 300] #64 500, num_nodes: Nodes for the LSTM cell
alpha = 0.0 #0.1 #0.1
beta = 0.0 #0.01 #10000.01
lambda_w = 0.00000001

dropout_cons = 1.0

Allow_Bias= True 

learning_rate= 0.0001      # 0.001
grad_clip_thresh= 0.001       # 0.00001

current_run= 2
batch_size= 64 #64
num_unrollings= 64 #100

batch_size_val= 64 #len(valid_text_pos)/num_unrollings_val #500

comment='_01alpha_noDropout_LcLpLcp_100unrol'

In [3]:
model_version = 'L'+str(len(num_nodes))
print("num_nodes:",num_nodes)
print("alpha:",alpha, ", beta:",beta,", lambda_w:", lambda_w)
print("num_unrollings:",num_unrollings, ", batch_size:",batch_size,", batch_size_val:", batch_size_val)
print("learning_rate:", learning_rate, 'grad_clip_thresh:', grad_clip_thresh)

num_nodes: [300, 300, 300]
alpha: 0.0 , beta: 0.0 , lambda_w: 1e-08
num_unrollings: 64 , batch_size: 64 , batch_size_val: 64
learning_rate: 0.0001 grad_clip_thresh: 0.01


# 1. Load training dataset

In [4]:
from vocabulary_coding_simple import *

vc= pickle.load( open( "imdb_vc.pkl", "rb" ) )
num_inputs=  vc.vocabulary_size
num_outputs= vc.vocabulary_size

text_pos= pickle.load( open( "imdb_text_pos.pkl", "rb" ) )
text_neg= pickle.load( open( "imdb_text_neg.pkl", "rb" ) )

In [5]:
print(vc.text2keys(['i','loved','the','movie']))
print(vc.keys2text([11,93,1,12]))

[11, 93, 1, 12]
i love the movi 


In [6]:
print('# inputs:',num_inputs)

len_text= min(len(text_pos), len(text_neg)) #CAREFUL
print('text length:',len_text)

# inputs: 1001
text length: 2264691


#### Creation of training, validation and testing sets

In [7]:
valid_size_pos = int(math.floor(len_text)*0.1)
valid_text_pos = text_pos[:valid_size_pos]
train_text_pos = text_pos[valid_size_pos:len_text] # excluding validation set
#train_text_pos = text_pos[:len_text]               # including validation set
train_size_pos = len(train_text_pos)

print(valid_size_pos)
print(train_size_pos)

226469
2038222


In [8]:
valid_size_neg = int(math.floor(len_text*0.1))
valid_text_neg = text_neg[:valid_size_neg]
train_text_neg = text_neg[valid_size_neg:len_text] # excluding validation set
#train_text_neg = text_neg[:len_text]               # including validation set
train_size_neg = len(train_text_neg)               
print(train_size_neg)

2038222


# 2. Batch generators

In [9]:
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10 # this is to prevent that log() returns minus infinity
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized
    probabilities.
    """
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def sample(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, vc.vocabulary_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p

def random_distribution():
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, vc.vocabulary_size])
    return b/np.sum(b, 1)[:,None]

Get supervised-batch from positive and negative texts

In [10]:
batches_pos = BatchGenerator(train_text_pos, batch_size//2, num_unrollings, vc)
batches_neg = BatchGenerator(train_text_neg, batch_size//2, num_unrollings, vc)

def GetNextBatch_sentiment(rand_shuffle= False):
    ''' Generates a batch of sentences and its corresponding labels for classification
        Returns:
            X: list of positive and negative sentences, the format is the same that the batches generated from the 
               positive and negative texts. len(X)= num_unrrolings, X[i].shape: [batch_size x vocabulary_size]
            y: class to which each sentence belong. The format is a vector of size [batch_size x 1]
    '''
    batch_pos = batches_pos.next()
    batch_neg = batches_neg.next()
        
    y_aux= np.zeros(shape=[batch_size,1], dtype=np.float) # y_aux= class for each one of the sentences
    y_aux[0:int(batch_size//2),0]= 1 # positive samples are denoted by 1, negative by 0
    
    # TODO: ADD MULTIPLE CLASSES
    if rand_shuffle:
        rand_ind= np.random.permutation(batch_size) # rand_ind= random shuffling for the sentences
        y_aux=y_aux[rand_ind,:]
    
    X= list()
    #y= list()
    # TODO: try to do not penalize mistakes being done in the first samples of the unrolling
    # start_ind= math.floor(0.5*len(batch_pos))
    for x_ind in range(len(batch_pos)):  # the length of batch_pos is the number of unrollings
        # Concatenate positive and negative batches and arrange them according to rand_ind
        if rand_shuffle:
            X.append( np.concatenate((batch_pos[x_ind],batch_neg[x_ind]),0)[rand_ind,:] )
        else:
            X.append( np.concatenate((batch_pos[x_ind],batch_neg[x_ind]),0) )
        
    return X, y_aux

batch_X, batch_y = GetNextBatch_sentiment()

### For Validation

In [11]:
num_unrollings_val= num_unrollings #10 #num_unrollings 
print('batch_size_val:', batch_size_val, ' suggested:', len(valid_text_pos)/num_unrollings_val)

batches_pos_val = BatchGenerator(valid_text_pos, batch_size_val//2, num_unrollings_val, vc)
batches_neg_val = BatchGenerator(valid_text_neg, batch_size_val//2, num_unrollings_val, vc)
#batches_pos_val = BatchGenerator(train_text_pos, batch_size_val/2, num_unrollings_val)
#batches_neg_val = BatchGenerator(train_text_neg, batch_size_val/2, num_unrollings_val)


def Get_Val():
    batch_pos = batches_pos_val.next()
    batch_neg = batches_neg_val.next()
        
    y_aux= np.zeros(shape=[batch_size_val,1], dtype=np.float) # y_aux= class for each one of the sentences
    y_aux[0:int(batch_size_val//2),0]= 1
    
    # TODO: ADD MULTIPLE CLASSES
    rand_ind= np.random.permutation(batch_size_val) # rand_ind= random shuffling for the sentences
    
    y_aux=y_aux[rand_ind,:]
    
    X= list()
    #y= list()
    # TODO: try to do not penalize mistakes being done in the first samples of the unrolling
    # start_ind= math.floor(0.5*len(batch_pos))
    for x_ind in range(len(batch_pos)):  # the length of batch_pos is the number of unrrolings
        # Concatenate positive and negative batches and arrange them according to rand_ind
        X.append( np.concatenate((batch_pos[x_ind],batch_neg[x_ind]),0)[rand_ind,:] )
        
    return X, y_aux

valid_X, valid_y = Get_Val()

batch_size_val: 64  suggested: 3538.578125


# 3. Model for positive and negative reviews

In [12]:
from lstm_model_V2 import *

class myLstmNet(LstmNet):
    def get_extra_inputs(self, i, h_list, state_list):
        #print('OK:', len(h_list))
        return i
    
    def evaluate_final_output(self, outputs_list, inputs_list, h_list ):
        ''' Calculates the final output of the neural network, usually it is just a linear transformation
        
        outputs_list: list with the outputs from the last lstm cell
        inputs_list: list of inputs to the network
        h_list: list with all hidden outputs from all the cells
        '''
        ''''''
        all_hidden = list()
        #print('n_unrollings:', len(h_list)) # DELETE !!!
        #print('n_layers:', len(h_list[0])) # DELETE !!!
        
        for t in h_list: # go trough each time step
            all_hidden.append( tf.concat(1,t) )
        return self.out_layer.evaluate(tf.concat(0, all_hidden))  
        
        
        # Original:
        #return self.out_layer.evaluate(tf.concat(0, outputs_list))
    
if len(num_nodes)>1:
    n_extra= [num_inputs for i in range(len(num_nodes)+1)]
    n_extra[0]= 0
    n_extra[-1]= sum(num_nodes) - num_nodes[-1]
else:
    n_extra= [0,0]

In [13]:
class ModelSetup:
    
    def __init__( self, pos_net, neg_net, w, b, batch_size, num_unrollings, drop_prob_list, name=''):
    
        # 1. Create placeholders for inputs 
        self.X = list()
        for iaux in range(num_unrollings + 1):
            self.X.append(tf.placeholder(tf.float32, shape=[batch_size, vc.vocabulary_size], 
                                             name= name+'X_i'+str(iaux)+'_All'))
        aux_inputs = self.X[:num_unrollings]
        aux_labels = self.X[1:]  # inputs shifted by one time step.

        # Create a list for store the placeholders for the labels
        self.labels = tf.placeholder(tf.float32, shape=[batch_size, 1])
       

        # -------------------- unrolling of the network --------------------------- # 
        self.pos_unroll, _= pos_net.unrolling_setup( batch_size, num_unrollings, 
                                                     inputs_list= aux_inputs,
                                                     labels_list= aux_labels,
                                                     drop_prob_list= drop_prob_list,
                                                     reset_between_unrollings= True,
                                                   )

        self.neg_unroll, _= neg_net.unrolling_setup( batch_size, num_unrollings, 
                                                     inputs_list= aux_inputs,
                                                     labels_list= aux_labels,
                                                     drop_prob_list= drop_prob_list,
                                                     reset_between_unrollings= True,
                                                   ) 


        # Classifier.
        y_pos= tf.reshape(self.pos_unroll.y, [-1, batch_size, num_outputs])
        y_neg= tf.reshape(self.neg_unroll.y, [-1, batch_size, num_outputs])
        
        y_pos_mean= tf.reduce_mean(y_pos, reduction_indices=0)
        y_neg_mean= tf.reduce_mean(y_neg, reduction_indices=0)
        
        output_mean= tf.concat(1, [ y_pos_mean, y_neg_mean ])

        self.logits = tf.nn.xw_plus_b( output_mean , w, b )
        
        self.error_per_sample= tf.nn.sigmoid_cross_entropy_with_logits( self.logits, self.labels )
        
        # prediction error
        #E_pos= tf.reshape(self.pos_unroll.error_per_sample,[num_unrollings,batch_size])  
        #E_neg= tf.reshape(self.neg_unroll.error_per_sample,[num_unrollings,batch_size]) 
        #
        #E_pos_mean= tf.reduce_mean(E_pos, reduction_indices= 0) 
        #E_neg_mean= tf.reduce_mean(E_neg, reduction_indices= 0)
        #
        #Lp = tf.reduce_mean( tf.mul(self.labels, output_pos_mean) + tf.mul(self.labels-1, output_neg_mean))
        #Lp = tf.reduce_mean( tf.mul(tf.squeeze(self.labels), E_pos_mean) + 
        #                     tf.mul(tf.squeeze(1-self.labels), E_neg_mean)
        #                   )
        # counter-prediction penalty
        #
        #Lcp = tf.reduce_mean( tf.mul(tf.squeeze(1-self.labels), tf.exp(-E_pos_mean)) + 
        #                      tf.mul(tf.squeeze(self.labels), tf.exp(-E_neg_mean))
        #                    )
        #
        #Lcp = tf.reduce_mean( tf.mul(tf.squeeze(1-self.labels), tf.reduce_mean(tf.exp(-E_pos), reduction_indices= 0) ) + 
        #                      tf.mul(tf.squeeze(self.labels), tf.reduce_mean(tf.exp(-E_neg), reduction_indices= 0) )
        #                    )
        
        # regularization
        l2_c= tf.nn.l2_loss(w)
        
        #self.loss = tf.reduce_mean( self.error_per_sample ) + alpha*Lp + beta*(1.0/Lcp)
        #self.loss = tf.reduce_mean( self.error_per_sample ) + alpha*Lp - beta*(Lcp)
        #self.alpha_r = tf.placeholder(tf.float32)
        #self.beta_r = tf.placeholder(tf.float32)
        
        #self.loss = tf.reduce_mean( self.error_per_sample ) #+ self.alpha_r*Lp + self.beta_r*Lcp + lambda_w*l2_c
        self.loss = tf.reduce_mean(self.error_per_sample) #+ lambda_w*l2_c

In [14]:

graph = tf.Graph()
with graph.as_default():
    # For dropout
    drop_prob = tf.placeholder(tf.float32)
    drop_prob_list = [ drop_prob for i in range(len(num_nodes)+1)]
    drop_prob_list[0]= None
     
    # 1. Define positive and negative neural networks
    pos_net= myLstmNet( num_inputs, num_nodes, num_outputs, n_extra= n_extra,
                        afunction=activation_function, 
                        LstmCell= AlexLstmCell,
                        name= pos_net_name)
        
    neg_net= myLstmNet( num_inputs, num_nodes, num_outputs, n_extra= n_extra,
                        afunction=activation_function, 
                        LstmCell= AlexLstmCell,
                        name= neg_net_name)
    
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_outputs*2, 1], -0.1, 0.1), name=('w_class')) # unconstrained w
    #w = tf.Variable(tf.constant( [[-2.0], [2.0]]), name=('w_class'), trainable=False) # fixed w
    
    #w = tf.Variable(tf.truncated_normal([1, 1], 0.3, 0.4), name=('w_class')) # unconstrained w
    #w = tf.Variable(tf.constant( [[0.3]] ), name=('w_class')) # fixed w
    #w = tf.concat(0, [-w, w])
    
    b = tf.Variable(tf.zeros([1]), name=('b_class'), trainable=Allow_Bias)
    
    
    
    # 2. Define unrolling for training
    train= ModelSetup( pos_net, neg_net, w, b, batch_size, num_unrollings, 
                       drop_prob_list, 
                       name='train_')
       
    # 3. Define unrolling for validation
    if allow_valid:
        valid= ModelSetup( pos_net, neg_net, w, b, batch_size_val, num_unrollings_val, 
                           drop_prob_list= [None for dummy in range(len(num_nodes))], 
                           name='valid_')

    # 4. Define unrolling for testing
    pos_gen_test, _= pos_net.unrolling_setup( 1, 1, drop_prob_list= [None, None, None, None, None] )
    neg_gen_test, _= neg_net.unrolling_setup( 1, 1, drop_prob_list= [None, None, None, None, None] )
    
          
    
    # 5. Define optimizer    
    # 1. specify the optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate) # ADAM 0.001
    
    # 2. get the gradients and variables
    # grads_and_vars is a list of tuples (gradient, variable). 
    grads_and_vars = optimizer.compute_gradients(train.loss) 
    gradients, v = zip(*grads_and_vars)
    
    # 3. process the gradients
    gradients, _ = tf.clip_by_global_norm(gradients, grad_clip_thresh)  #1.25 #0.025 #0.001(last used)
    # 4. apply the gradients to the optimization procedure
    optimizer = optimizer.apply_gradients( zip(gradients, v) ) # ADAM
    
    # for prediction
    train_pred = tf.nn.sigmoid(train.logits)
    
    # Saver
    saver = tf.train.Saver()

# 4. Train Energy predictor model

In [15]:
reload_pos= False
load_pos_file=  working_dir+"weights/Weights_LSTM_" + model_version +"_"+ str(num_nodes[0]) + "u_"+ \
                "1r_pos.ckpt"
    
reload_neg= False
load_neg_file= working_dir+"weights/Weights_LSTM_" + model_version +"_"+ str(num_nodes[0]) + "u_"+ \
                "1r_neg.ckpt"

reload_all= True
load_all_file=  working_dir+"weights/Weights_LSTM_" + model_version +"_"+ str(num_nodes[0]) + "u_"+ \
                str(current_run-1) +"r_All"+comment+".ckpt";

save_all= True
save_all_file=  working_dir+"weights/Weights_LSTM_" + model_version +"_"+ str(num_nodes[0]) + "u_"+ \
                str(current_run) +"r_All"+comment+".ckpt";

In [ ]:
if current_run==1:
    reload_all= False;
else:
    reload_pos= False;
    reload_neg= False;

In [ ]:
num_steps = 3000 
summary_frequency = 50
n_valid_tests = 50
n_characters_step= num_unrollings*batch_size

aux_print=0
with tf.Session(graph=graph) as session:
    # -------------------------------- Load weigths from file, or initialize variables ------------------------------------
    if reload_all:
        saver.restore(session, load_all_file)
        #session.run( global_step.assign(0) ) # SGD
        
    elif (reload_pos and reload_neg):
        tf.initialize_all_variables().run()
        pos_net.saver.restore(session, load_pos_file)
        neg_net.saver.restore(session, load_neg_file)
        print('Weights for positive and negative networks loaded')
    else:
        tf.initialize_all_variables().run()
        print('Weights Initialized')
    
    # ------------------------------------------- Training loop --------------------------------------------------
    print('step | Train_err, Train_mis | Valid_err, Valid_mis | loss')
    
    mean_loss = 0.0
    mean_mis_pred = 0.0
    for step in range(num_steps):
        # 1. Get next batch
        batch_X, batch_y = GetNextBatch_sentiment()        
        
        # 2. Setup feed dictionary for network 1 and network 2
        feed_dict = dict()
        # For dropout
        feed_dict[drop_prob] = dropout_cons
        # hyperparameters
        '''
        if step<200:
            feed_dict[train.alpha_r] = 1.0
            feed_dict[train.beta_r] = 0.0
        else:
            feed_dict[train.alpha_r] = alpha
            feed_dict[train.beta_r] = beta
        '''    
        # Inroduce labels
        feed_dict[train.labels] = batch_y
        # Introduce inputs
        for i in range(num_unrollings+1):
            feed_dict[train.X[i]] = batch_X[i]
        
        
        
        # 3 Run optimizer
        #_, l, lr = session.run( [optimizer, loss_train, learning_rate], feed_dict=feed_dict) # SGD
        _, l, pred_train_aux = session.run( [optimizer, train.loss, train_pred], feed_dict=feed_dict) # ADAM
        
        mean_mis_pred += np.sum( np.not_equal( np.greater(pred_train_aux, 0.5), np.greater(batch_y, 0.5)) )
        mean_loss += l
        
        # --------------------------------------------- logging ------------------------------------------------
        if train_size_pos<aux_print :
            print('+'*80)
            print('')
            aux_print = 0
        else:
            aux_print += n_characters_step
        
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
                mean_mis_pred = mean_mis_pred / summary_frequency
            
            '''train mis classification'''        
            prediction_train = session.run( [tf.nn.sigmoid(train.logits)], feed_dict=feed_dict)          
            mis_pred_train = np.sum( np.not_equal( np.greater(prediction_train, 0.5), np.greater(batch_y, 0.5)) )
            
            # ---------- Print loss in validation dataset ---------
            if allow_valid:
                '''valid mis classification'''
                mis_pred_val = 0
                for step_valid in range(n_valid_tests):
                    # 1. Get next batch
                    batch_X_val, batch_y_val = Get_Val()        
                    feed_dict_val = dict()
                    # For dropout
                    feed_dict_val[drop_prob] = 1.0
                    # Inroduce labels
                    feed_dict_val[valid.labels] = batch_y_val
                    # Introduce inputs
                    for i in range(num_unrollings_val+1):
                        feed_dict_val[valid.X[i]] = batch_X_val[i]
                    # 2. Get prediction
                    prediction_val = session.run( [tf.nn.sigmoid(valid.logits)], feed_dict=feed_dict_val)          
                    mis_pred_val = mis_pred_val*step_valid
                    mis_pred_val += np.sum( np.not_equal( np.greater(prediction_val, 0.5), np.greater(batch_y_val, 0.5)) )
                    mis_pred_val = mis_pred_val/(step_valid+1)
                    #Delete from here
                    #print('err=', np.sum( np.not_equal( np.greater(prediction_val, 0.5), np.greater(batch_y_val, 0.5)) ))
                    #to here
                    
                    #if ((mis_pred_val/batch_y_val.shape[0])>0.2) and (step_valid==0):
                    #    break                        
                    #elif (step_valid==0):
                    #    break
                    #    print('Low validation error reached:', (mis_pred_val/batch_y_val.shape[0]))
                
                ''' weights and bias for classficator '''
                # print values for b and w
                b_t= b.eval()
                w_t= w.eval()
                
                print('%d | %f (%f), %d | %f, %d | %f | w:[%f,%f], b:%f' % (step, 
                                                     mis_pred_train/batch_y.shape[0], mean_mis_pred/batch_y.shape[0], mis_pred_train,
                                                     mis_pred_val/batch_y_val.shape[0], mis_pred_val,
                                                     mean_loss,
                                                     w_t[0], w_t[1], b_t[0]
                                                    ))    
                #if(mis_pred_val/batch_y_val.shape[0] < 0.20):
                #    break
            else:
                print('%d | %f, %f | %f' % (step, mis_pred_train/batch_y.shape[0], mis_pred_train, mean_loss))
                      
            mean_loss = 0
            mean_mis_pred = 0
            
    if save_all:
        save_path = saver.save(session, save_all_file)
        print("LSTM cell Model for negative reviews saved in file: %s" % save_path)
        

step | Train_err, Train_mis | Valid_err, Valid_mis | loss
0 | 0.203125 (0.250000), 13 | 0.240937, 15 | 0.555719 | w:[-0.063232,-0.123253], b:-0.000049
50 | 0.281250 (0.249375), 18 | 0.256875, 16 | 0.506943 | w:[-0.063232,-0.123253], b:-0.000049
100 | 0.265625 (0.248438), 17 | 0.220938, 14 | 0.497953 | w:[-0.063232,-0.123253], b:-0.000049
150 | 0.234375 (0.244062), 15 | 0.270625, 17 | 0.488840 | w:[-0.063232,-0.123253], b:-0.000048
200 | 0.218750 (0.219375), 14 | 0.229687, 14 | 0.460236 | w:[-0.063233,-0.123254], b:-0.000048
250 | 0.171875 (0.230313), 11 | 0.253437, 16 | 0.489941 | w:[-0.063233,-0.123254], b:-0.000048
300 | 0.250000 (0.234063), 16 | 0.247500, 15 | 0.487042 | w:[-0.063233,-0.123254], b:-0.000048
350 | 0.296875 (0.233750), 19 | 0.249062, 15 | 0.476599 | w:[-0.063233,-0.123254], b:-0.000048
400 | 0.234375 (0.251250), 15 | 0.251250, 16 | 0.498981 | w:[-0.063233,-0.123254], b:-0.000048
450 | 0.218750 (0.241875), 14 | 0.243750, 15 | 0.499500 | w:[-0.063233,-0.123254], b:-0.00

## Generate some samples from positive and negative networks

In [ ]:
with tf.Session(graph=graph) as session:
    # Load weigths from file, or initialize variables
    tf.initialize_all_variables().run()
    saver.restore(session, save_all_file)
    print('Weights loaded')
        
    # print values for b and w
    b_t= b.eval()
    w_t= w.eval()
    #b_t= session.run(b)
    print('b=', b_t)
    print('w=', w_t)
    
    
    # For positive network
    print('-'*30 + 'pos' + '-'*30)
    for _ in range(5):
        feed = sample(random_distribution())
        sentence = vc.prob2char(feed)
        pos_gen_test.reset_saved_out_state.run()
        for _ in range(70):
            prediction = tf.nn.softmax(pos_gen_test.y).eval({pos_gen_test.inputs_list[0]: feed}) # feed is a 1-hot encoding
            feed = sample(prediction)  # sample returns a 1-hot encoding
            sentence += vc.prob2char(feed)
        print(sentence)
    print('-' * 80)
    
    # For negative network
    print('-'*30 + 'neg' + '-'*30)
    for _ in range(5):
        feed = sample(random_distribution())
        sentence = vc.prob2char(feed)
        neg_gen_test.reset_saved_out_state.run()
        for _ in range(70):
            prediction = tf.nn.softmax(neg_gen_test.y).eval({neg_gen_test.inputs_list[0]: feed}) # feed is a 1-hot encoding
            feed = sample(prediction)  # sample returns a 1-hot encoding
            sentence += vc.prob2char(feed)
        print(sentence)
    print('=' * 80)